# Aggregazioni con Pandas

## Module 4: Aggregazioni
- Utilizzo di `.crosstab` per contare un conteggio di frequenza per ogni abbinamento di categorie
- Utilizzo di `.pivot_table` per calcolare aggregati di valori numerici per ogni abbinamento di categorie (come una tabella pivot del foglio di calcolo)

In [27]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/emilianoyoulysses/corso_python_fondazione_pico/refs/heads/DataFrames-%26-TimeSeries/MpaW6ovPRg6Je5MueztN_tips.csv")
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# Cos'è `.crosstab?`
- Crosstab calcola una semplice tabulazione incrociata di due (o più) fattori
- Calcola una tabella di frequenza dei fattori
- Esempio: contare quanti tavoli hanno pranzato o cenato ogni giorno?
- Esempio: contare il numero di tavoli per fumatori suddivisi per genere?

In [28]:
df.day.unique() # vogliamo tutti i giorni diversi

array(['Sun', 'Sat', 'Thur', 'Fri'], dtype=object)

In [29]:
df.time.unique() # tutti i diversi tempi

array(['Dinner', 'Lunch'], dtype=object)

In [6]:
df[(df.day == "Thur") & (df.time == "Lunch")].shape[0] # conto il pranzo giovedì, indicizzazione composta

61

In [30]:
df[(df.day == "Thur") & (df.time == "Dinner")].shape[0] # ripeto per ogni combinazione giorno/ora

1

### In alternativa

In [31]:
df[df.day == "Thur"].time.value_counts() # eseguo .time.value_counts() ogni singolo giorno su specifica colonna

time
Lunch     61
Dinner     1
Name: count, dtype: int64

In [32]:
pd.crosstab(index=df.day, columns=df.time) # conteggio le frequenze di tutti i giorni per tutti i tempi 

time,Dinner,Lunch
day,,
Fri,12,7
Sat,87,0
Sun,76,0
Thur,1,61


In [10]:
pd.crosstab(index=df.day, columns=df.time, margins=True) # Margins=True mostra i totali di riga/colonna

time,Dinner,Lunch,All
day,,,
Fri,12,7,19
Sat,87,0,87
Sun,76,0,76
Thur,1,61,62
All,176,68,244


In [11]:
# Normalize=True mostra percentuali invece di conteggi grezzi
pd.crosstab(index=df.day, columns=df.time, margins=True, normalize=True).round(2)

time,Dinner,Lunch,All
day,,,
Fri,0.05,0.03,0.08
Sat,0.36,0.00,0.36
Sun,0.31,0.00,0.31
Thur,0.00,0.25,0.25
All,0.72,0.28,1.00


In [12]:
# Possiamo anche passare elenchi di serie in indici o colonne
# gli argomenti indice e colonne possono tenere una lista di due o più colonne differenti categoricalmente. 
pd.crosstab(index=df.day, columns=[df.time, df.smoker])

time   Dinner     Lunch    
smoker     No Yes    No Yes
day                        
Fri         3   9     1   6
Sat        45  42     0   0
Sun        57  19     0   0
Thur        1   0    44  17

## Utilizzo di pivot_tables per aggregare più dei conteggi
Utilizziamo `.pivot_table` per impostare gli abbinamenti di categoria, quindi specifica la colonna da misurare, in forma aggregata e le tue funzioni aggregate

- Il metodo `.pivot_table` utilizza per impostazione predefinita la media,

- Possiamo specificare più categorie nell'indice e nelle colonne, ma i risultati possono diventare visivamente occupati

- Esempio: per ogni abbinamento giorno/ora, calcola la media `total_bill`

- Esempio: per ogni abbinamento giorno/ora, ottieni la media `total_bill` e `tip`

- Esempio: per ogni accoppiamento giorno/ora, calcola il min, la mediana, il max `tip`

In [35]:
print(df.dtypes)


total_bill    float64
tip           float64
sex            object
smoker         object
day            object
time           object
size            int64
dtype: object


In [39]:
import pandas as pd 
# pivot lavora sull'intero DataFrame a differenza di cross che lavora su una serie o più serie alla volta
# Senza specificare una colonna "valori"
# pivot_table restituisce la media numerica delle colonne numeriche, suddivisa per ogni coppia di categorie

# Usa l'argomento valori per specificare le colonne numeriche
# average_total_bill

pd.pivot_table(df, index="day", columns="time", values="total_bill")

time,Dinner,Lunch
day,,
Fri,19.663333,12.845714
Sat,20.441379,NaN
Sun,21.410000,NaN
Thur,18.780000,17.664754


Average number size è il numero di persone per tavolo.

In [16]:
# Usa l'argomento "valori" per specificare quali colonne calcolare
pd.pivot_table(df, index="day", columns="time", values=["total_bill", "tip"])

tip           total_bill           
time    Dinner     Lunch     Dinner      Lunch
day                                           
Fri   2.940000  2.382857  19.663333  12.845714
Sat   2.993103       NaN  20.441379        NaN
Sun   3.255132       NaN  21.410000        NaN
Thur  3.000000  2.767705  18.780000  17.664754

In [17]:
# Usa l'argomento aggfunc per sovrascrivere la funzione media predefinita
pd.pivot_table(df, values="tip", aggfunc="median", index="day", columns="time")

time,Dinner,Lunch
day,,
Fri,3.00,2.2
Sat,2.75,NaN
Sun,3.15,NaN
Thur,3.00,2.3


In [18]:
#L'argomento aggfunc può prendere un elenco di funzioni aggregate
pd.pivot_table(df, values="tip", aggfunc=["min", "median", "max"], index="day", columns="time")

min       median          max      
time Dinner Lunch Dinner Lunch Dinner Lunch
day                                        
Fri    1.00  1.58   3.00   2.2   4.73  3.48
Sat    1.00   NaN   2.75   NaN  10.00   NaN
Sun    1.01   NaN   3.15   NaN   6.50   NaN
Thur   3.00  1.25   3.00   2.3   3.00  6.70

## Esercizi

- Usa la tabella incrociata sul dataframe `tips` per contare il numero di tabelle di dimensioni diverse per ogni momento della giornata. *Suggerimento* ricorda che `.size` è un attributo integrato sugli oggetti panda.

- Usa `pd.read_csv` e il file `mpg.csv` per creare un dataframe denominato `mpg`.

- Usa `.crosstab` per contare il numero di veicoli per ogni combinazione di classe e trasmissione. *Suggerimento* ricorda che `class` è una parola riservata in Python.

- Usa `.crosstab` per contare il numero di veicoli per ogni combinazione di produttore e trasmissione.

- Usa `.pivot_table` e `mpg` per calcolare il chilometraggio medio dell'autostrada per ogni combinazione di classe di veicolo e trasmissione.

- Usa `.pivot_table` e `mpg` per calcolare il chilometraggio medio della città per ogni combinazione di produttore e trasmissione.

In [34]:
pd.crosstab(index=df["size"], columns=df.time)

time,Dinner,Lunch
size,,
1,2,2
2,104,52
3,33,5
4,32,5
5,4,1
6,1,3


In [22]:
mpg = pd.read_csv("https://raw.githubusercontent.com/emilianoyoulysses/corso_python_fondazione_pico/refs/heads/DataFrames-%26-TimeSeries/mpg.csv")
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
0,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
1,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
2,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
3,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
4,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
229,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
230,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
231,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
232,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


In [23]:
pd.crosstab(index=mpg['class'], columns=mpg.drv)


drv,4,f,r
class,,,
2seater,0,0,5
compact,12,35,0
midsize,3,38,0
minivan,0,11,0
pickup,33,0,0
subcompact,4,22,9
suv,51,0,11


In [24]:
pd.crosstab(index=mpg.manufacturer, columns=mpg.drv)

drv,4,f,r
manufacturer,,,
audi,11,7,0
chevrolet,4,5,10
dodge,26,11,0
ford,13,0,12
honda,0,9,0
hyundai,0,14,0
jeep,8,0,0
land rover,4,0,0
lincoln,0,0,3


In [25]:
pd.pivot_table(mpg, values='hwy', index='class', columns='drv')

drv,4,f,r
class,,,
2seater,NaN,NaN,24.800000
compact,25.833333,29.142857,NaN
midsize,24.000000,27.552632,NaN
minivan,NaN,22.363636,NaN
pickup,16.878788,NaN,NaN
subcompact,26.000000,30.545455,23.222222
suv,18.274510,NaN,17.454545


In [26]:
pd.pivot_table(mpg, aggfunc="median", values="cty", index="manufacturer", columns="drv" )

drv,4,f,r
manufacturer,,,
audi,17.0,18.0,NaN
chevrolet,12.5,18.0,14.5
dodge,12.5,16.0,NaN
ford,13.0,NaN,15.0
honda,NaN,24.0,NaN
hyundai,NaN,18.5,NaN
jeep,14.0,NaN,NaN
land rover,11.5,NaN,NaN
lincoln,NaN,NaN,11.0


# Esercitazione
Imparare a usare `pd.crosstab()` e `df.pivot_table()` per riepilogare e analizzare i dati con tabelle incrociate e medie raggruppate.

In [1]:
import pandas as pd

df = pd.DataFrame([
    {"modello": "Iveco Daily", "alimentazione": "diesel", "categoria": "medio", "anno": 2022, "noleggio_giornaliero": 100},
    {"modello": "Mercedes Actros", "alimentazione": "diesel", "categoria": "pesante", "anno": 2021, "noleggio_giornaliero": 230},
    {"modello": "Fiat Ducato", "alimentazione": "benzina", "categoria": "leggero", "anno": 2022, "noleggio_giornaliero": 80},
    {"modello": "Tesla Semi", "alimentazione": "elettrico", "categoria": "pesante", "anno": 2023, "noleggio_giornaliero": 210},
    {"modello": "Ford Transit", "alimentazione": "diesel", "categoria": "medio", "anno": 2021, "noleggio_giornaliero": 90},
    {"modello": "Renault Master", "alimentazione": "elettrico", "categoria": "leggero", "anno": 2023, "noleggio_giornaliero": 95}
])


 1. Conta camion per alimentazione e categoria
 2. Aggiungi anche il totale per riga e colonna
 3. Conta camion per anno e categoria
 
 4. Calcola il noleggio medio per categoria
 5. Noleggio medio per categoria e alimentazione
 6. Aggiungi una funzione di aggregazione diversa (es. max)
 7. Noleggio medio per anno e categoria
 

In [2]:
pd.crosstab(df["alimentazione"], df["categoria"])

categoria,leggero,medio,pesante
alimentazione,,,
benzina,1,0,0
diesel,0,2,1
elettrico,1,0,1


In [3]:
pd.crosstab(df["alimentazione"], df["categoria"], margins=True)

categoria,leggero,medio,pesante,All
alimentazione,,,,
benzina,1,0,0,1
diesel,0,2,1,3
elettrico,1,0,1,2
All,2,2,2,6


In [4]:
pd.crosstab(df["anno"], df["categoria"])

categoria,leggero,medio,pesante
anno,,,
2021,0,1,1
2022,1,1,0
2023,1,0,1


In [5]:
df.pivot_table(values="noleggio_giornaliero", index="categoria")

,noleggio_giornaliero
categoria,
leggero,87.5
medio,95.0
pesante,220.0


In [6]:
df.pivot_table(values="noleggio_giornaliero", index="categoria", columns="alimentazione")

alimentazione,benzina,diesel,elettrico
categoria,,,
leggero,80.0,NaN,95.0
medio,NaN,95.0,NaN
pesante,NaN,230.0,210.0


In [7]:
df.pivot_table(values="noleggio_giornaliero", index="categoria", columns="alimentazione", aggfunc="max")

alimentazione,benzina,diesel,elettrico
categoria,,,
leggero,80.0,NaN,95.0
medio,NaN,100.0,NaN
pesante,NaN,230.0,210.0


In [8]:
df.pivot_table(values="noleggio_giornaliero", index="anno", columns="categoria")

categoria,leggero,medio,pesante
anno,,,
2021,NaN,90.0,230.0
2022,80.0,100.0,NaN
2023,95.0,NaN,210.0
